In [1]:
#import statements
import pandas as pd
import numpy as np
import time
import os

import spacy

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPool1D
from tensorflow.keras.optimizers import Adam

In [2]:
df = pd.read_csv('input_data_base.csv', index_col = 0)

In [4]:
df.head()

,Trigger document,Earmarked Funds,Document item,Business Transaction,Consptn in Doc.Crncy,Currency,Revaluation rate,Consumption in LC,Local Currency,Delete Date,Document Date,Reduction text,Purchasing Document,Reference document,Reference procedure,Reference line item,Item,Line item,Seq. No. of Account Assgt,GL
1,0014268084 00020,NaN,1.0,Purchase Requisition,0,CAD,NaN,0,CAD,2018-03-21,2018-03-21,Freight Charges,14268084.0,14268084.0,PREQ,20.0,20.0,NaN,NaN,2201.0
2,0014263945 00020,NaN,1.0,Purchase Requisition,0,CAD,NaN,0,CAD,2018-03-27,2018-03-19,Freight charges,14263945.0,14263945.0,PREQ,20.0,20.0,NaN,NaN,2201.0
3,0014263837 00020,NaN,1.0,Purchase Requisition,0,CAD,NaN,0,CAD,2018-03-28,2018-03-19,Freight,14263837.0,14263837.0,PREQ,20.0,20.0,NaN,NaN,2201.0
4,0014262000 00020,NaN,1.0,Purchase Requisition,0,CAD,NaN,0,CAD,2018-03-27,2018-03-16,Freight Charges,14262000.0,14262000.0,PREQ,20.0,20.0,NaN,NaN,2201.0
5,0014260003 00030,NaN,1.0,Purchase Requisition,0,CAD,NaN,0,CAD,2018-03-28,2018-03-15,Freight,14260003.0,14260003.0,PREQ,30.0,30.0,NaN,NaN,2201.0


In [6]:
working_data = df[['GL','Reduction text',]]

In [7]:
working_data.head()

,GL,Reduction text
1,2201.0,Freight Charges
2,2201.0,Freight charges
3,2201.0,Freight
4,2201.0,Freight Charges
5,2201.0,Freight


In [12]:
#balance data by GL

working_data.groupby('GL').count()
entries = min(list(working_data.groupby('GL').count()['Reduction text']))
balanced_GLs = []
index_list = list(working_data.groupby('GL').count().index)
for GL in index_list:
    balanced_GLs.append(working_data[working_data['GL']==GL].sample(entries))

balanced_data = pd.concat(balanced_GLs,axis = 0, ignore_index = True)


In [10]:
#define and import functions for NLP

nlp = spacy.load('en_vectors_web_lg')
def similarityy(x,y):
    return(nlp(x).similarity(nlp(y)))

def sim_comp(col,word):
    out = []
    col = list(col)
    for entry in col:
        out.append(similarityy(entry,word))
    return(out)


In [14]:
#word list for comparison, create word similary matrix


a2201 = 'freight'
b2201 = 'handling'
c2201 = 'delivery'

a7208 = 'paint'
b7208 = 'glue'
c7208 = 'spray'

a7209 = 'clothing'
b7209 = 'protection'
c7209 = 'glove'

a7210 = 'hardware'
b7210 = 'stock'
c7210 = 'metal'

a7312 = 'abrasive'
b7312 = 'grind'
c7312 = 'sand'

a7341 = 'lube'
b7341 = 'oil'
c7341 = 'hydraulic'

a9129 = 'tool'
b9129 = 'machine'
c9129 = 'hand'

a9219 = 'car part'
b9219 = 'engine'
c9219 = 'pipe'

a9220 = 'tire'
b9220 = 'rubber'
c9220 = 'wheel'

a9335 = 'battery'
b9335 = 'volt'

#create df with simili
ready_df = pd.DataFrame(columns = ['GL', 'Reduction text', a2201, b2201, c2201, a7208, b7208, c7208, a7209, b7209, c7209, a7210, b7210, c7210, a7312, b7312, c7312, a7341, b7341, c7341, a9129, b9129, c9129, a9219, b9219, c9219, a9220, b9220, c9220, a9335, b9335])
ready_df['GL'] = balanced_data['GL']
ready_df['Reduction text'] = balanced_data['Reduction text']

#freight
ready_df[a2201] = sim_comp(balanced_data['Reduction text'], a2201)
ready_df[b2201] = sim_comp(balanced_data['Reduction text'], b2201)
ready_df[c2201] = sim_comp(balanced_data['Reduction text'], c2201)

#paint lube oil
ready_df[a7208] = sim_comp(balanced_data['Reduction text'], a7208)
ready_df[b7208] = sim_comp(balanced_data['Reduction text'], b7208)
ready_df[c7208] = sim_comp(balanced_data['Reduction text'], c7208)

#PPE
ready_df[a7209] = sim_comp(balanced_data['Reduction text'], a7209)
ready_df[b7209] = sim_comp(balanced_data['Reduction text'], b7209)
ready_df[c7209] = sim_comp(balanced_data['Reduction text'], c7209)

#metals + hardware
ready_df[a7210] = sim_comp(balanced_data['Reduction text'], a7210)
ready_df[b7210] = sim_comp(balanced_data['Reduction text'], b7210)
ready_df[c7210] = sim_comp(balanced_data['Reduction text'], c7210)

#abrasive materials
ready_df[a7312] = sim_comp(balanced_data['Reduction text'], a7312)
ready_df[b7312] = sim_comp(balanced_data['Reduction text'], b7312)
ready_df[c7312] = sim_comp(balanced_data['Reduction text'], c7312)
 
# POL hydraulic
ready_df[a7341] = sim_comp(balanced_data['Reduction text'], a7341)
ready_df[b7341] = sim_comp(balanced_data['Reduction text'], b7341)
ready_df[c7341] = sim_comp(balanced_data['Reduction text'], c7341)

# tools
ready_df[a9129] = sim_comp(balanced_data['Reduction text'], a9129)
ready_df[b9129] = sim_comp(balanced_data['Reduction text'], b9129)
ready_df[c9129] = sim_comp(balanced_data['Reduction text'], c9129)

#parts
ready_df[a9219] = sim_comp(balanced_data['Reduction text'], a9219)
ready_df[b9219] = sim_comp(balanced_data['Reduction text'], b9219)
ready_df[c9219] = sim_comp(balanced_data['Reduction text'], c9219)

#tire
ready_df[a9220] = sim_comp(balanced_data['Reduction text'], a9220)
ready_df[b9220] = sim_comp(balanced_data['Reduction text'], b9220)
ready_df[c9220] = sim_comp(balanced_data['Reduction text'], c9220)
 
#battery
ready_df[a9335] = sim_comp(balanced_data['Reduction text'], a9335)
ready_df[b9335] = sim_comp(balanced_data['Reduction text'], b9335)

#ready_df.to_csv('ready_data.csv')
df = ready_df.copy()

c:\users\olfoa\appdata\local\programs\python\python36\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
c:\users\olfoa\appdata\local\programs\python\python36\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
c:\users\olfoa\appdata\local\programs\python\python36\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
c:\users\olfoa\appdata\local\programs\python\python36\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
c:\users\olfoa\appdata\local\programs\python\python36\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
c:\users\olfoa\appdata\local\programs\python\python36\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
c:\u

In [16]:
#make outputs integer
df = ready_df.copy()
A = list(df['GL'])

for i in range(len(A)):
    A[i] = int(str(A[i])[:4])

df['GL'] = A

LNum_GL={1:2201,2:7208,3:7209,4:7210,5:7312,6:7341,7:9129,8:9219,9:9220,10:9335}
LNums = list(LNum_GL.keys())
GLs = list(LNum_GL.values())
GL_LNum = dict(zip(GLs, LNums))

def GL2LNum(col):
    for i in range(len(col-1)):
        col[i]=GL_LNum[(col[i])]
    return(col)

df['GL'] = GL2LNum(df['GL'])

c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#pre-process data for ANN
#ready_df.to_csv('ready_data.csv')
x = df.drop(['GL','Reduction text'], axis = 1)
y = df['GL']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.15, random_state = 0, stratify = y)
x_train = np.array(x_train)
x_test = np.array(x_test)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
#build ANN

model = Sequential()
model.add(Conv1D(32, 11, activation='relu', input_shape = x_train[0].shape))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv1D(64, 11, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(11, activation='sigmoid'))

from tensorflow.keras.optimizers import Adam
model.compile(optimizer = Adam(lr = 0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
epochs = 20
history = model.fit(x_train, y_train, epochs = epochs, validation_data = (x_test, y_test), verbose = 1)

In [17]:
#execution text
#must specify model 
import pandas as pd
import spacy
import tensorflow as tf
from tensorflow import keras
model = tf.keras.models.load_model('2020-03-07-1535-Conv1D.model')
nlp = spacy.load('en_vectors_web_lg')
import numpy as np

def similarityy(x,y):
    return(nlp(x).similarity(nlp(y)))

def sim_comp(col,word):
    out = []
    col = list(col)
    for entry in col:
        out.append(similarityy(entry,word))
    return(out)

#convert input must be a COLUMN
def convert_input(text):
    a2201 = 'freight'
    b2201 = 'handling'
    c2201 = 'delivery'

    a7208 = 'paint'
    b7208 = 'glue'
    c7208 = 'spray'

    a7209 = 'clothing'
    b7209 = 'protection'
    c7209 = 'glove'

    a7210 = 'hardware'
    b7210 = 'stock'
    c7210 = 'metal'

    a7312 = 'abrasive'
    b7312 = 'grind'
    c7312 = 'sand'

    a7341 = 'lube'
    b7341 = 'oil'
    c7341 = 'hydraulic'

    a9129 = 'tool'
    b9129 = 'machine'
    c9129 = 'hand'

    a9219 = 'car part'
    b9219 = 'engine'
    c9219 = 'pipe'

    a9220 = 'tire'
    b9220 = 'rubber'
    c9220 = 'wheel'

    a9335 = 'battery'
    b9335 = 'volt'

    ready_df = pd.DataFrame(columns = [a2201, b2201, c2201, a7208, b7208, c7208, a7209, b7209, c7209, a7210, b7210, c7210, a7312, b7312, c7312, a7341, b7341, c7341, a9129, b9129, c9129, a9219, b9219, c9219, a9220, b9220, c9220, a9335, b9335])

    #freight
    ready_df[a2201] = sim_comp(text, a2201)
    ready_df[b2201] = sim_comp(text, b2201)
    ready_df[c2201] = sim_comp(text, c2201)

    #paint lube oil
    ready_df[a7208] = sim_comp(text, a7208)
    ready_df[b7208] = sim_comp(text, b7208)
    ready_df[c7208] = sim_comp(text, c7208)

    #PPE
    ready_df[a7209] = sim_comp(text, a7209)
    ready_df[b7209] = sim_comp(text, b7209)
    ready_df[c7209] = sim_comp(text, c7209)

    #metals + hardware
    ready_df[a7210] = sim_comp(text, a7210)
    ready_df[b7210] = sim_comp(text, b7210)
    ready_df[c7210] = sim_comp(text, c7210)

    #abrasive materials
    ready_df[a7312] = sim_comp(text, a7312)
    ready_df[b7312] = sim_comp(text, b7312)
    ready_df[c7312] = sim_comp(text, c7312)

    # POL hydraulic
    ready_df[a7341] = sim_comp(text, a7341)
    ready_df[b7341] = sim_comp(text, b7341)
    ready_df[c7341] = sim_comp(text, c7341)

    # tools
    ready_df[a9129] = sim_comp(text, a9129)
    ready_df[b9129] = sim_comp(text, b9129)
    ready_df[c9129] = sim_comp(text, c9129)

    #parts
    ready_df[a9219] = sim_comp(text, a9219)
    ready_df[b9219] = sim_comp(text, b9219)
    ready_df[c9219] = sim_comp(text, c9219)

    #tire
    ready_df[a9220] = sim_comp(text, a9220)
    ready_df[b9220] = sim_comp(text, b9220)
    ready_df[c9220] = sim_comp(text, c9220)

    #battery
    ready_df[a9335] = sim_comp(text, a9335)
    ready_df[b9335] = sim_comp(text, b9335)
    return(ready_df)

#makes it in proper shaped array
def convert_input2(input):
    input = np.array(convert_input(input))
    return(input.reshape(input.shape[0], input.shape[1], 1))

def text_to_df(text):
    dumy = pd.DataFrame(columns = ['Reduction text'])
    dumy['Reduction text'] = [text]
    return(dumy['Reduction text'])

def predict_GL(text):
    a = text_to_df(text)
    b = convert_input2(a)
    c = model.predict_classes(b)
    Num_2Desc={1:'2201 - freight', 2:'7208 - paints, chemicals, coating', 3:'7209 - personal protective equipment', 4:'7210 - materials and hardware', 5:'7312 - abrasives', 6:'7341 - POL products', 7:'9129 - tools', 8:'9219 - repair parts',9:'9220 - tires', 10:'9335 - batteries'}
    d = c[0]
    e = Num_2Desc[d]
    return(e)

In [18]:
predict_GL('filter') # correct

'9219 - repair parts'

In [19]:
predict_GL('oil filter') #wrong, should be 9219 repair parts

'7341 - POL products'

In [20]:
predict_GL('metal stock') #correct

'7210 - materials and hardware'